# TD3 - Exploration de l'historique des prénoms en France

Nous allons enfin passer à une véritable application du Data Mining dans ce TD. Suite à la préparation des données dans le TD précédent, nous allons en effet pouvoir explorer les grandes tendances de ce jeu puis nous chercherons à déceler quelques particularités remarquables cachées sous la masse des données en entrée.

Ce TD est exclusivement dédié à la famille d'application `Description` du Data Mining. Le prochain TD portera quant à lui sur l'explication (modélisation d'une série temporelle) et la structuration (*clustering*). L'associativité, ou recherche des motifs fréquents, ne sera pas abordée en TD faute de temps.

## Etude des grandes tendances

Commençons tout d'abord par s'intéresser aux grandes tendances de ce jeu de données, en partant d'une simple analyse univariée. Nous étudierons la distribution du nombre d'individu portant un même prénom, toutes années et toutes régions confondues. Nous conserverons la variable qualitative `sexe` puisque l'on s'attend à observer des différences de distribution selon les deux sexes.

In [ ]:
# Lecture du jeu traité précédemment

import pandas as pd

df = pd.read_csv(r'prenoms_transformed.csv', index_col=0, dtype={
    'sexe': 'category',
    'prenom': 'category',
    'gender': 'category',})

df = df.astype({'region': 'category'})

df.head()

In [ ]:
# Calcul des effectifs pour chaque prénom donné

effectifs_par_prenoms = df.groupby(['prenom', 'sexe'], observed=True, as_index=False).nombre.sum()

effectifs_par_prenoms.sample(5) # 5 effectifs au hasard

In [ ]:
# Diagramme de distribution de cette série
import plotly.express as px

px.histogram(effectifs_par_prenoms, x='nombre', y='nombre', 
             histfunc='count', # nous aurons en ordonnées "count(nombre)"
             log_y=True, 
             facet_col='sexe', 
             color='sexe',
             marginal='rug',
             labels={'nombre': 'Effectif total'},
             title='Distribution des effectifs pour tous les prénoms donnés depuis 1900')

Remarquons que plotly trace par défaut des histogrammes de classes d'amplitudes égales. Nous pourrions choisir de trace des classes d'effectifs égaux en calculant d'abord les quantiles souhaités du jeu de données.

On remarque comme prévu que l'immense majorité des prénoms sont relativement peu donnés en France (l'échelle des ordonnées est logarithmique !). De surcroît, la repartition des effectifs est légèrement plus étalée pour les hommes que pour les femmes, indiquant que les prénoms féminins sont généralement plus rares que les prénoms féminins. 

On constate cependant l'existance d'un prénom singulier pour les femmes (`MARIE`), qui est remarquablement courant, bien plus que le prénom masculin le plus courant (`JEAN`).

En guise d'exercice, on pourrait s'intéresser pour chaque sexe à la proportion des effectifs cumulés pour les 5 ou 10 prénoms les plus courants par rapport aux effectifs cumulés de tous les autres prénoms. En d'autres termes, on s'intéressent dans ce cas au niveau de domination des prénoms courants par rapport aux prénoms moins courants.

Pour poursuivre cette étude, on pourrait ensuite s'intéresser aux effectifs cumulés :
* Des 5 prénoms les plus courants
* Du 6è au 100è prénom le plus courant
* De tous les autres prénoms

Dans les deux cas, des diagrammes camembert sont adapté en raison du faible nombre de classes à représenter, et du rôle de la représentation (importance d'une classe par rapport à l'ensemble).

Ces deux exercices sont laissés à la charge du lecteur.

In [ ]:
# Effectifs cumulés du top 5 vs les autres (pour chaque sexe)



In [ ]:
# Effectifs cumulés du top 5, du top 100 et des autres (pour chaque sexe)



## Exploration multivariée des données

Outre l'étude des grandes tendances, rappelons que l'exploration de données est **un processus avant tout créatif**. Libre à vous donc de **définir des axes intéressants d'études**.

En pratique il est fréquent d'avoir des **préjugés ou des a priori** sur un phénomène ou un système que l'on connaît ou maîtrise (ou pense maîtriser). Ces a priori fournissent naturellement les premiers axes d'analyse, le but étant de les vérifier en toute **impartialité** à l'aide des données disponibles.

En ce qui concerne le cas de ce TD, mes *a priori* initiaux sont les suivants, du plus certain au moins sûr :
* le fichier des prénoms à la naissance fournit une bonne image de la natalité française, l'impact des guerres et de la transition démographique sera facilement visible
* il y a davantage de garçons qui naîssent que de filles, et cet écart est biologique donc doit rester approximativement constant entre les départements
* il y a de plus en plus de prénoms différents donnés chaque année, en particulier depuis les années 80
* les hommes reçoivent davantage de prénoms composés que les femmes
* les prénoms composés deviennent de plus en plus rares au fil du temps
* les nouveaux prénoms apparaissant chaque année sont en majorité des dérivés de prénoms existants avec une légère modification orthographique. En pratique la proportion de chaque groupe de prénoms (ex : Alexandre, Alexandra, Alex, Alexis, etc...) est semblable à la proportion du prénom original (Alexandre) observée au début du siècle.
* Une partie disproportionnée des prénoms commencent par une lettre proche du début de l'alphabet
* Les prénoms typiquement régionaux (e.g. bretons, corses, basques, etc...) sont devenus progressivement impopulaires au début du XXe siècle mais redeviennent de plus en plus courants récemment (XXIe siècle)


Nous allons approfondir quelques-uns de ces différents axes d'étude. Ceux n'étant pas abordés constituent de bons exercices laissés à la charge du lecteur.

Les différents choix de visualisation sont avant tout personnels et en aucun cas la "bonne" solution. **D'autres graphes ou choix graphiques peuvent être plus appropriés**.

### Etude de la natalité

In [ ]:
grouped = df.groupby(['annee', 'sexe'], as_index=False).nombre.sum()

grouped.head()

In [ ]:
# Tracé de l'évolution de la natalité

px.line(grouped, x='annee', y='nombre', color='sexe')

L'impact des deux guerres mondiales est saisissant. On observe également le Baby Boom s'étalant de 1945 à 1975. Ce fichier semble donc peindre une bonne image de la natalité en France.

On peut également étudier l'évolution de la natalité d'une année sur l'autre. Pour ce faire, concentrons nous sur les différences de natalité N/N-1 et traçons-les sous forme de diagramme en barre. Une simple échelle de couleur est associée à ce graphe pour renforcer sa compréhension (vert : natalité en hausse, rouge : en baisse).

In [ ]:
nat_diff = grouped.copy()
nat_diff['evolution'] = grouped.nombre.diff(2)
nat_diff = nat_diff.dropna()
nat_diff = nat_diff.astype({'evolution': 'int'})

nat_diff.head()

In [ ]:
fig = px.bar(nat_diff, x='annee', y='evolution', 
             color=pd.np.sign(nat_diff.evolution),
             facet_col='sexe', 
             labels={'evolution': 'Evolution naissance N/N-1','annee': 'Année'},
             color_continuous_scale='RdYlGn' # Echelle de couleur Rouge → Vert
      )

fig.update_layout(coloraxis_showscale = False) # On cache la légende

Remarque : Un [diagramme Renko](https://datavizproject.com/data-type/renko-chart/) superposé au premier graphe ci-dessus (lineplot) est une alternative élégante, quoique plus difficile à produire en Python, à ce deuxième graphe.

Avec Plotly, vous pourrez le produire assez simplement avec le type de graphe [Waterfall](https://plot.ly/python/waterfall-charts/). L'utilisation de ce graphe est laissée en exercice.

### Ecart garçon/fille à la naissance

Nous étudions dans cette partie le [sex ratio](https://fr.wikipedia.org/wiki/Sex-ratio), *i.e.* la proportion homme/femme à la naissance. Ce sex ratio est normalement très légèrement supérieur à 1 et n'est réellement affecté que par des pratiques sociétales (infanticide, avortement après avoir pris connaissance du sexe de l'enfant, etc...). Le stress intense serait également un facteur favorisant la diminution du sex ratio (la population devient alors plus féminine).

Vérifions l'évolution de cette statistique à travers les âges en France.

In [ ]:
# Partons de l'aggrégation précédente
grouped.head()

Nous devons "pivoter" ce tableau pour obtenir deux colonnes correspondant aux deux modalités de `sexe`, avec l'année en index et les effectifs en tant que valeur.

Cette opération de "pivotage" (ou tableau croisé sur Excel) peut être effectuée dans pandas avec la fonction `pivot`. Cette dernière accepte la colonne à utiliser pour créer les nouvelles colonnes, la colonne à utiliser pour calculer le nouvel index, et la colonne à utiliser pour calculer les nouvelles valeurs.

In [ ]:
pd.pivot_table(grouped, columns='sexe', index='annee', values='nombre', aggfunc='sum')

In [ ]:
pivoted = grouped.pivot(columns='sexe', # Créer une colonne par modalité au sein de la colonne "sexe"
                        index='annee',  # Utiliser les valeurs de cette colonne comme index
                        values='nombre',# Utiliser les effectifs totaux comme valeur à conserver
                       ) 

pivoted.head()

In [ ]:
# La colonne "sexe" était qualitative, donc les colonnes créées le sont également
# Un bug dans pandas empêche la création de nouvelles colonnes si ces dernières sont qualitatives
# Nous corrigeons leur type pour palier temporairement à ce bug
pivoted.columns = pivoted.columns.astype(str)

# Nous pouvons à présent calculer simplement le sex ratio de chaque année
pivoted['sex_ratio'] = pivoted.Homme / pivoted.Femme
pivoted.head()

In [ ]:
# Traçons enfin l'évolution du sex ratio

px.line(pivoted, y='sex_ratio', x=pivoted.index)

La courbe précédente indique un sex ratio actuel (à la naissance) de 1.14 ce qui classerait la France nettement en tête du [classement](https://fr.wikipedia.org/wiki/Liste_des_pays_par_taux_de_masculinité). On peut donc affirmer que ces données sont visiblement incorrectes en l'état. Parmi les raisons possibles on peut citer :
* Le tronquage des données brutes en raison de démarches d'anonymisation
* La suppression de lignes opérée lors du TD précédent pour améliorer la qualité du jeu de données
* La réattribution de la modalité `_PRENOMS_RARES` lors du précédent TD qui a pu perturber l'équilibre H/F

On ne pourra donc pas tirer d'enseignement de cette première exploration, passons donc à la suite.

### Evolution du nombre de prénoms uniques

In [ ]:
unique_hf = df.groupby(['annee', 'sexe']).agg(nombre_prenoms_uniques=('prenom', 'nunique')).reset_index()

unique_hf.head()

In [ ]:
px.line(unique_hf,
        x='annee',
        y='nombre_prenoms_uniques', 
        color='sexe',
        color_discrete_sequence=['Red', 'Blue'],
        labels={'nombre_prenoms_uniques': 'Nombre de prénoms uniques',
                'annee': 'Année'})

Il y a toujours eu une plus grande diversité dans les prénoms féminins que masculins, mais dans les deux cas la tendance est à la hausse. Cette croissance s'est accélérée dans les années 2000 jusqu'à se stabiliser il y a peu.

On remarque de surcroît que l'écart de diversité entre les deux sexes tend à ce réduire. Plusieurs phénomènes peuvent possiblement expliquer ceci :
* Les prénoms masculins sont de plus en plus diversifiés, ce qui se traduit par un nombre de prénoms uniques accrus
* Les prénoms féminins sont de moins en moins diversifiés

Pour séparer plus clairement ces deux phénomènes possibles, vous pourrez vous intéresser à l'évolution N/N-1 pour chaque sexe et tracer par exemple la différence H/F de cette évolution. Ceci revient mathématiquement à calculer la dérivée de chaque série et étudier la différence des dérivées.

Cette étude est laissée en exercice.

### Evolution de l'attribution des prénoms composés

On définit un prénom composé comme tout prénom possédant au moins un trait d'union (`-`).

Commençons par les filtrer depuis le jeu de donnée global.

In [ ]:
df_prenoms_composes = df[df.prenom.str.contains('-')]
df_prenoms_composes.head()

Afin de tester deux axes d'analyses, nous allons extraire les pourcentages d'attributions des prénoms composés par année et par sexe. Le pourcentage est calculé par rapport au nombre total d'individus nés cette année avec le même sexe.

Nous représenterons enfin ce DataFrame sous la forme d'un diagramme à aires empilées. Afin de mieux discerner les séries "Homme" et "Femme", nous tracerons en réalité la série "Homme" et l'opposé de la série "Femme". Cette astuce de calcul nous permet alors de générer simplement un [Stream Graph](https://datavizproject.com/data-type/stream-graph/).

In [ ]:
grouped_prenoms_composes = df_prenoms_composes.groupby(['annee', 'sexe'], as_index=False).nombre.sum()
grouped_prenoms_composes.head()

In [ ]:
ratio_prenoms_composes = grouped_prenoms_composes.copy()
ratio_prenoms_composes['ratio'] = ratio_prenoms_composes.nombre / grouped.nombre
ratio_prenoms_composes.head()

In [ ]:
# On passe les ratio pour les Femmes à l'opposé (multiplication par -1) pour inverser le graphe
ratio_prenoms_composes.loc[ratio_prenoms_composes.sexe =='Femme', 'ratio'] *= - 1
ratio_prenoms_composes.head()

In [ ]:
fig = px.area(ratio_prenoms_composes, x='annee', y='ratio', color='sexe', 
              color_discrete_sequence=['Red', 'Blue'],
              labels={'annee': 'Année',
                      'ratio': '% prénom composé / total prénoms'})

# Nous remplissons l'intérieur de chaque trace uniquement jusqu'au 0 des ordonnées

fig.update_traces({'fill': 'tozeroy'})
# fig.update_traces({'fill': 'tonexty'}) # Valeur par défaut, décommentez cette ligne pour observer la différence

Les prénoms composés apparaissent clairement comme étant en perte de vitesse depuis les années 60, sans aucun signe de regain d'intérêt.

Au sommet de leur popularité, les hommes recevaient davantage de prénoms composés que les femmes, mais ce ne fut pas toujours le cas, notamment au début du XXe siècle où les proportions étaient presque identiques.

Les prénoms composés restent dans tous les cas de figure minoritaires, représentant à l'apogée de leur popularité autour de 10% des prénoms attribués (hommes et femmes cumulés) en terme d'effectif.

Remarque : pour mieux mettre en valeur le changement de proportion de l'affectation de prénoms composés entre hommes et femmes (davantage les femmes au début du XXe siècle puis davantage les hommes), un diagramme à aires empilées trié ([**sorted streamgraph**](https://datavizproject.com/data-type/sorted-stream-graph/)) peut se révéler utile. Leur génération restent toutefois délicates avec les bibliothèques utilisées ici (et en général dans la plupart des outils).

### Popularité des prénoms régionaux

Il convient de définir avant tout le concept de prénoms "régionaux". On propose deux solutions :
* Tout prénom défini dans une liste externe de prénoms régionaux, on se charge alors de vérifier la qualité de cette source externe avant de l'utiliser pour repérer parmi tous les prénoms ceux étant issus d'une identité régionale (par exemple avec la fonction `isin`)
* De manière plus automatisée, on considère un prénom comme régional si sur l'ensemble des effectifs portant ce prénom, plus de la moîtié d'entre eux sont issus d'une région en particulier. On peut se fixer également un effectif cumulé minimum afin d'être certain de conserver des prénoms populaires (car leur effectif cumulé total est important) et principalement affectés à une région en particulier.

On choisit d'appliquer la deuxième méthode pour ne pas avoir à trouver une autre source de données.

Pour tenir compte des écarts naturels de naissances entre les régions, on choisit de procéder à deux transformations :
1. Normaliser chaque effectif pour chaque couple `(prenom, region)` pour le nombre de naissance total dans la région
2. Normaliser une seconde fois les effectifs réduits précédents pour chaque couple `(prenom, region)` par la somme des effectifs réduits pour ce prénom

Après la deuxième normalisation, la somme des effectifs normalisés par chaque prénom doit donc valoir 1.

In [ ]:
# Premier groupby : calculer les effectifs cumulés par prénom et par région
grouped_par_region = df.groupby(['prenom', 'region']).nombre.sum()

print(len(grouped_par_region))
grouped_par_region.head()

In [ ]:
# On filtre les prénoms dont l'effectif cumulé (toutes régions confondues) ne dépasse pas un seuil fixé
seuil = 50 # arbitraire

grouped_par_region = grouped_par_region.groupby('prenom').filter(lambda x: x.sum() > seuil)

print(len(grouped_par_region))
grouped_par_region.head(10)

In [ ]:
# Transformation 1 : 
# On réduit pour chaque prénom les effectifs par le nombre total de naissances dans la région
# Ceci permet de placer tous les prénoms sur la même échelle et de prendre en compte la différence de population
# entre les régions
grouped_par_region_normalise = grouped_par_region.groupby(['region', 'prenom']).sum() / df.groupby('region').nombre.sum()
grouped_par_region_normalise.head(10)

In [ ]:
# Transformation 2 : 
# Pour chaque prénom, normaliser les effectifs par l'effectif total du prénom (somme pour toutes les régions)
distrib_prenoms_courants = grouped_par_region_normalise.groupby('prenom').transform(lambda x: x / x.sum())
distrib_prenoms_courants.head(10)

In [ ]:
# Deuxième filtre : on conserve uniquement les prénoms dont l'effectif réduit dans une région vaut au moins 50%
distrib_prenoms_regionaux = distrib_prenoms_courants.groupby('prenom').filter(lambda x: x.max() >= 0.5)
distrib_prenoms_regionaux.head(10)

Toutes les données nécessaires sont désormais calculées. On peut vérifier que ce mode de sélection automatique fonctionne en vérifiant pour quelques régions les prénoms reconnus comme régionaux les plus donnés (en relatif).

In [ ]:
distrib_prenoms_regionaux['Bretagne'].nlargest(10)

In [ ]:
distrib_prenoms_regionaux['Corse'].nlargest(10)

Ce mode de sélection semble conforme à l'intuition.

On peut donc extraire la liste de tous les prénoms régionaux et observer leur évolution au fil du temps.

In [ ]:
prenoms_regionaux = distrib_prenoms_regionaux.reset_index().prenom

prenoms_regionaux.head()

In [ ]:
px.area(df[df.prenom.isin(prenoms_regionaux)].groupby('annee', as_index=False).nombre.sum(),
        x='annee', y='nombre',
        labels={'annee': 'Année',
                'nombre': 'Nombre de prénoms régionaux'})

Conformément à l'a priori, les prénoms régionaux ont connu un certain passage à vide au milieu du siècle dernier mais sont plus populaires que jamais.

Afin de visualiser les régions contribuants le plus à ces effectifs de prénoms régionaux, traçons un diagramme à aires empilées où chaque aire représente l'évolution des effectifs cumulés des prénoms régionaux pour chaque région.

In [ ]:
# On récupère les prénoms régionaux pour chaque région
prenom_regionaux_par_region = distrib_prenoms_regionaux[distrib_prenoms_regionaux >= 0.5]

In [ ]:
# On a donc les couples (region, prenom) à conserver
# Pour récupérer les effectifs il faut repartir du jeu de données complet, que l'on indexe avec les mêmes couples (region, prenom)
tmp = df.set_index(['region' , 'prenom']).loc[prenom_regionaux_par_region.index, ['annee', 'nombre']]

tmp.head()

In [ ]:
# Enfin, on veut les effectifs cumulés pour tous les prénoms régionaux par région et année

tmp = tmp.groupby(['region', 'annee']).nombre.sum().reset_index()

tmp.head(10)

In [ ]:
# Traçons enfin notre diagramme à aires empilées
px.area(tmp, x='annee', y='nombre', color='region')

Ce graphe est riche en enseignements. On peut notamment citer l'explosion de nouveaux prénoms en Île-de-France, la baisse progressive des prénoms régionaux en Auvergne-Rhône-Alpes et des Hauts-de-France, ou encore l'absence notable de certaines régions pourtant frontalières telles l'Occitanie.

Rappelons qu'un prénom a été défini comme régional si 
* il existe au moins une année où il a été majoritairement donné dans une unique région
* au moins 50 personnes en cumulés ont possédé ce prénom (pour éviter les modalités rares, les variantes orthographiques, etc...)

Nous pourrions étudier le pourcentage de partage de ces prénoms entre chaque région au fil des années. Il est alors nécessaire de construire le tableau de contigence entre chaque paire de régions (une matrice 14x14) comptant le nombre de prénoms en commun. On pourra représenter cette matrice par une carte thermique (*heatmap*) ou un [diagramme en cordes](https://datavizproject.com/data-type/chord-diagram/).

Entraînez-vous à produire le tableau de contingence ci-dessous avec la fonction `pivot_table` ou `crosstab`.

In [ ]:
# Calcul du tableau de contingence
# Compter le nombre de modalités en commun (ou l'effectif cumulé lié à ces modalités) pour chaque couple de région



### Mouvement de population


Pour cette dernière analyse, nous allons nous baser sur les données supplémentaires intégrées lors du TD précédent. Afin de pouvoir espérer observer une manifestation des différentes vagues d'immigration en France au cours du XXè siècle, nous allons étudier la statistique suivante calculée pour chaque prénom et chaque année : `effectif_cumule * abs(frequence_region - frequence_France)`.

Cette statistique peut être vue comme une image de l'affectation d'un prénom donné dans chaque grande région du monde avec les effectifs français. On s'intéresse à `abs(frequence_region - frequence_France)` afin d'étudier la dissimilarité des prénoms par rapport aux coutumes de nommage françaises.

Une fois cette statistique calculée pour chaque couple `(annee, prénom)`, nous conservons la médiane de ces statistiques pour chaque année afin de la tracer.

In [ ]:
df.head()

In [ ]:
# Créons une liste contenant les noms de colonnes avec les fréquences d'attribution dans d'autres régions

international_cols = ['europe_est', 'europe_ouest', 'europe_nord', 'europe_sud', 'asie_ouest', 'asie_centre' , 'asie_est']
df[international_cols]

In [ ]:
# Première aggrégation des effectifs par couple (prénom, année)
# On inclus également toutes les fréquences d'affectation dans ce groupby
# Ces fréquences sont uniques à chaque prénom donc ne changent pas l'agrégation

effectifs_cumules = df.groupby(['prenom', *international_cols, 'France', 'annee'], 
                               observed=True).nombre.sum().reset_index()

effectifs_cumules.head()

In [ ]:
# Puis calculons la statistique souhaitée en lieu et place des fréquences d'affectation

effectifs_cumules[international_cols] = (effectifs_cumules[international_cols]
                                         .apply(lambda x: abs(x - effectifs_cumules.France) * effectifs_cumules.nombre, 
                                                axis=0))

effectifs_cumules.head()

In [ ]:
# Agrégeons une dernière fois ces lignes uniquement par année
# On conserve ici la médiane pour ne pas être trop affecté par les extrêmes

by_year = effectifs_cumules.groupby('annee')[international_cols].median().reset_index()

by_year.head()

Ce dernier DataFrame n'est pas dans le format `tidy` : les noms de colonnes contiennent une variable (ici le nom de la région concernée).

Nous le transformons dans ce format en le "dépivotant". Dans pandas, le plus simple est d'utiliser la fonction `melt` qui accepte :
* les colonnes à conserver telles quelles (ici `annee`)
* les colonnes à dépivoter (ici toutes sauf `annee`)
* le nom de la colonne qui contiendra les noms des colonnes dépivotées (ici `region_monde`)
* le nom de la colonne qui contiendra les valeurs initiales (ici `mediane_affectation`)

In [ ]:
to_plot = pd.melt(by_year, 
                  id_vars = 'annee',
                  value_vars=international_cols,
                  var_name = 'region_monde', 
                  value_name='mediane_affectation')

to_plot.head()

On peut enfin passer à tracé de la figure.

In [ ]:
fig = px.area(to_plot, 
              x='annee',
              y='mediane_affectation',
              color='region_monde',
              line_shape='spline', # Lissage des courbes pour améliorer le rendu
              facet_row='region_monde', 
              height=1200, 
              labels={'mediane_affectation': 'Médiane affectation'})

fig.update_layout(showlegend=False) # Masquer la légende de droite
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])) # Nettoyer le titre de chaque graphe

On peut observer quelques grandes tendances sur ce graphe, notamment les vagues d'immigration en provenance de l'Europe du Sud d'abord dans les années 20 puis post-seconde Guerre Mondiale.

On remarquera également pendant la seconde Guerre Mondiale exclusivement un pic de prénoms (et donc de naissances) en provenance d'Europe de l'Ouest mais dissimilaires aux prénoms typiquement Français. En d'autres termes, il s'agit de prénoms courants en Europe de l'Ouest mais pas en France. La baisse subite de ces naissances dès 1945 laisse suggérer qu'il s'agit possiblement d'enfants nés de soldats étrangers présents en France.

Remarquons toutefois que l'étude des prénoms donnés à la naissance ne permet qu'une appréciation tronquée des vagues d'immigration. En effet, les naissances ont nécessairement lieu après une arrivée en France, possiblement des années plus tard. De plus, le prénom donné à l'enfant n'a pas nécessairement de raison d'être "typique" de la région d'origine de ses parents.

# Conclusion

Ce premier TD vous a permis d'appréhender les **principaux concepts et difficultés du Data Mining**. Certains types d'analyse n'ont pas été explorés ici, en particulier les données géographiques laissées à l'écart ou encore les données sur la phonétique inutilisées également.

Les visualisations employées sont assez simplistes. Ceux désirant explorer d'autres visualisations sont invités à simplement importer les données traités dans un logiciel approprié (Tableau, PowerBI, etc...) ou à poursuivre leur apprentissage de plotly.

Il est enfin à noter que cette analyse répond bien à la définition du Data Mining en cela qu'elle a permis de dégager des concepts haut-niveau, facilement assimilables par un humain, à partir d'un jeu de donnée difficilement interprétable tel quel (3.5M de lignes).

Un rapport d'analyse sur ce jeu de donnée s'appuierait sur les différents graphes et métriques produits dans ce notebook afin de tirer des conclusions pertinentes sur le jeu de donnée. Dans le cas précis, une mise en relation avec divers phénomènes historiques (*e.g.* évolution du droit des femmes) permettrait de donner un début d'explication aux phénomènes observés.